In [1]:
import hoomd
from hoomd.md import bond, angle
import gsd
import gsd.hoomd
import numpy

def run_one(k=500):
    device = hoomd.device.CPU()
    # device = hoomd.device.GPU()
    simulation = hoomd.Simulation(device = device)

    #initialize 3 atoms
    N = 3
    L = 5
    frame = gsd.hoomd.Frame()
    frame.configuration.box = [L, L, L, 0, 0, 0]

    #make three particles
    frame.particles.N = N
    frame.particles.position = [[-.55,.55,0],[0,0,0],[.55,.55,0]]
    frame.particles.mass = [1.0] * N
    frame.particles.types = ['A']
    frame.particles.typeID = [0] * N

    #set two bonds
    frame.bonds.N=2
    frame.bonds.types = ['A-A']
    frame.bonds.typeID = [0]*2
    frame.bonds.group = numpy.array([[0,1],[1,2]])

    #and an angle
    frame.angles.N=1
    frame.angles.types = ['A-A-A']
    frame.angles.typeID = [0]
    frame.angles.group = numpy.array([[0,1,2]])
    
    with gsd.hoomd.open(name='smol.gsd', mode='a') as f:
        f.append(frame)
    simulation.create_state_from_gsd(filename='smol.gsd')

    harmonic = bond.Harmonic()
    harmonic.params['A-A'] = dict(k=k, r0=1.5)
    angles = angle.Harmonic()
    angles.params['A-A-A'] = dict(k=600, t0=1.7)
    kT = 1.
    ends = hoomd.filter.Tags([0,2])
    simulation.state.thermalize_particle_momenta(filter=ends, kT=kT)

    integrator = hoomd.md.Integrator(dt = 0.005)
    nvt = hoomd.md.methods.Langevin(kT = kT, filter = hoomd.filter.Tags([2]),default_gamma = 1.0)
    integrator.methods.append(nvt)
    simulation.operations.integrator = integrator
    gsd_writer = hoomd.write.GSD(trigger=hoomd.trigger.Periodic(10), filename="traj.gsd", mode='wb', filter=hoomd.filter.All())
    simulation.operations.writers.append(gsd_writer)
    simulation.run(1e4)

run_one(5000)

*Warning*: Simulation.seed is not set, using default seed=0
*Warning*: Simulation.seed is not set, using default seed=0
